In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import time
import pickle
import cv2
%matplotlib inline
import matplotlib.image as mpimg

import tensorflow
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model, load_model 
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Input
from tensorflow.keras import backend as k 
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from skimage.transform import resize

In [2]:
#feature extraction

In [3]:
def data_to_features(model,path,out):
    feature_dict={0:[],1:[]}
    classes=['0','1']
    for i in classes:
        l=os.listdir(path+i+'/')
        for j in l:
            im = cv2.imread(path+i+'/'+j)
            im = cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), (224, 224)).astype(np.float32) / 255.0
            im = np.expand_dims(im, axis =0)
            feature_dict[int(i)].append(model.predict(im))
    pickle.dump(feature_dict, open(out, "wb" ))
    return feature_dict

In [4]:
model=load_model('mal_ftr_extrn_model_30eph.h5')
model_till_flatten = Model(inputs=model.input,outputs=model.get_layer('flatten').output)
model_till_fc1 = Model(inputs=model.input,outputs=model.get_layer('fc1').output)

In [5]:
feature_dict0= data_to_features(model_till_flatten,'E:/ds project/mal_mc_dataset/train/' ,'features_till_flatten.pkl')

In [6]:
feature_dict1=data_to_features(model_till_fc1, 'E:/ds project/mal_mc_dataset/train/','features_till_fc1.pkl')

In [7]:
feature_dict0[0][0].shape

(1, 25088)

In [8]:
feature_dict1[0][0].shape

(1, 4096)

In [9]:
feature_dict2=data_to_features(model_till_flatten,'E:/ds project/mal_mc_dataset/validation/',
                               'features_till_flatten_validation.pkl')

In [10]:
feature_dict2[0][0].shape

(1, 25088)

In [11]:
#svm

In [12]:
labels = []
feature_list = []
feature_dict=pickle.load(open('features_till_flatten_validation.pkl','rb'))
names = [0,1]
for cat,features  in feature_dict.items():
    labels.extend([int(cat)] * len(features))
    feature_list.append(features)
feature_list = np.concatenate(feature_list)

In [13]:
feature_dict

{0: [array([[0.50480455, 0.        , 0.10855022, ..., 0.        , 0.73825103,
          0.        ]], dtype=float32),
  array([[0.5789035 , 0.        , 0.15648228, ..., 0.        , 0.6977023 ,
          0.        ]], dtype=float32),
  array([[0.47652334, 0.        , 0.04772666, ..., 0.        , 0.7464223 ,
          0.        ]], dtype=float32),
  array([[0.45887956, 0.        , 0.        , ..., 0.        , 0.7387006 ,
          0.        ]], dtype=float32),
  array([[0.47652334, 0.        , 0.04772666, ..., 0.        , 0.7464223 ,
          0.        ]], dtype=float32),
  array([[0.5624435 , 0.        , 0.14920288, ..., 0.        , 0.73037344,
          0.        ]], dtype=float32),
  array([[0.5624435 , 0.        , 0.14920288, ..., 0.        , 0.73037344,
          0.        ]], dtype=float32),
  array([[0.42772517, 0.        , 0.        , ..., 0.        , 0.7299413 ,
          0.        ]], dtype=float32),
  array([[0.4689726, 0.       , 0.       , ..., 0.       , 0.6863597,
       

In [14]:
feature_list1=[]
for i in range(0,len(feature_list)):
    feature_list1.append(feature_list[i].ravel())

In [15]:
clf = SVC(kernel='rbf', probability=True).fit(feature_list1,labels )

In [16]:
count=0
for i in feature_list:
    print("Prediction : "+str(clf.predict(i)[0]),end='')
    print("   Actual is : "+str(labels[count]))
    count=count+1

Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Prediction : 0   Actual is : 0
Predicti

In [17]:
#svm-full

In [18]:
labels = []
feature_list = []
feature_dict=pickle.load(open('features_till_flatten.pkl','rb'))
names = [0,1]
for cat,features  in feature_dict.items():
    labels.extend([int(cat)] * len(features))
    feature_list.append(features)
feature_list = np.concatenate(feature_list)

In [19]:
feature_list1=[]
for i in range(0,len(feature_list)):
    feature_list1.append(feature_list[i].ravel())

In [20]:
feature_list1[0].shape

(25088,)

In [21]:
scaler = StandardScaler().fit(feature_list1)
rescaledX = scaler.transform(feature_list1)
num_folds = 10
seed = 7
kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
model = SVC()
results = cross_val_score(model, rescaledX, labels)
print ("Accuracy: {} ({})".format(results.mean()*100.0, results.std()*100.0))

Accuracy: 98.9338824067719 (1.648084371359818)


In [22]:
scoring = 'accuracy'

c_values = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3, 1.5, 1.7, 2.0]
kernel_values = [ 'linear' , 'poly' , 'rbf' , 'sigmoid' ]
param_grid = dict(C=c_values, kernel=kernel_values)

model = SVC()

kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)

grid_result = grid.fit(rescaledX, labels)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 1.000000 using {'C': 0.1, 'kernel': 'linear'}
1.000000 (0.000000) with: {'C': 0.1, 'kernel': 'linear'}
1.000000 (0.000000) with: {'C': 0.1, 'kernel': 'poly'}
0.989177 (0.010824) with: {'C': 0.1, 'kernel': 'rbf'}
1.000000 (0.000000) with: {'C': 0.1, 'kernel': 'sigmoid'}
1.000000 (0.000000) with: {'C': 0.3, 'kernel': 'linear'}
1.000000 (0.000000) with: {'C': 0.3, 'kernel': 'poly'}
0.997826 (0.006522) with: {'C': 0.3, 'kernel': 'rbf'}
1.000000 (0.000000) with: {'C': 0.3, 'kernel': 'sigmoid'}
1.000000 (0.000000) with: {'C': 0.5, 'kernel': 'linear'}
1.000000 (0.000000) with: {'C': 0.5, 'kernel': 'poly'}
0.997826 (0.006522) with: {'C': 0.5, 'kernel': 'rbf'}
1.000000 (0.000000) with: {'C': 0.5, 'kernel': 'sigmoid'}
1.000000 (0.000000) with: {'C': 0.7, 'kernel': 'linear'}
1.000000 (0.000000) with: {'C': 0.7, 'kernel': 'poly'}
0.997826 (0.006522) with: {'C': 0.7, 'kernel': 'rbf'}
1.000000 (0.000000) with: {'C': 0.7, 'kernel': 'sigmoid'}
1.000000 (0.000000) with: {'C': 0.9, 'kernel': 'line

In [23]:
tuned_model = SVC(C=0.1, kernel='rbf')

In [24]:
tuned_model.fit(rescaledX, labels)

SVC(C=0.1)

In [25]:
labels2 = []
feature_list = []
feature_dict=pickle.load(open('features_till_flatten_validation.pkl','rb'))
names = [0,1]
for cat,features  in feature_dict.items():
    labels2.extend([int(cat)] * len(features))
    feature_list.append(features)
feature_list = np.concatenate(feature_list)

In [26]:
feature_list2=[]
for i in range(0,len(feature_list)):
    feature_list2.append(feature_list[i].ravel())

In [27]:
rescaledX_validation = scaler.transform(feature_list2)

result = tuned_model.score(rescaledX_validation, labels2)
print('Accuracy: %.3f%%' % (result * 100.0))

Accuracy: 97.403%


In [28]:
#svm

In [29]:
#preparing feature extraction whole new dataset

In [30]:
def data_to_features(model,path,out):
    feature_dict={0:[],1:[]}
    classes=['0','1']
    for i in classes:
        l=os.listdir(path+i+'/')
        for j in l:
            im = cv2.imread(path+i+'/'+j)
            im = cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), (224, 224)).astype(np.float32) / 255.0
            im = np.expand_dims(im, axis =0)
            feature_dict[int(i)].append(model.predict(im))
    pickle.dump(feature_dict, open(out, "wb" ))
    return feature_dict

In [31]:
#for whole data

In [32]:
feature_dict_new_ds=data_to_features(model_till_flatten,'E:/ds project/mmlr_images_act/',
                               'features_till_flatten_new_ds.pkl')

In [33]:
feature_dict_new_ds.keys()

dict_keys([0, 1])

In [34]:
#coverting dict 2D array to 1D array only uninfected data that is folder 0

In [35]:
uninfected = np.squeeze(feature_dict_new_ds[0])
uninfected

array([[0.5399395 , 0.        , 0.16691294, ..., 0.        , 0.7013653 ,
        0.        ],
       [0.56019056, 0.        , 0.1644939 , ..., 0.        , 0.6946348 ,
        0.        ],
       [0.5613014 , 0.        , 0.16069959, ..., 0.        , 0.75673145,
        0.        ],
       ...,
       [0.51160204, 0.        , 0.11073983, ..., 0.        , 0.73198646,
        0.        ],
       [0.5488362 , 0.        , 0.13346636, ..., 0.        , 0.73435974,
        0.        ],
       [0.51160204, 0.        , 0.11073983, ..., 0.        , 0.73198646,
        0.        ]], dtype=float32)

In [36]:
#storing it in DataFrame

In [37]:
uninfected_df =  pd.DataFrame(uninfected)
uninfected_df

,0,1,2,3,4,5,6,7,8,9,...,25078,25079,25080,25081,25082,25083,25084,25085,25086,25087
0,0.539940,0.0,0.166913,0.279902,0.653289,0.0,0.0,0.0,0.0,0.128804,...,1.269654,0.139180,0.265222,0.093659,0.0,0.205243,0.0,0.0,0.701365,0.0
1,0.560191,0.0,0.164494,0.302933,0.644150,0.0,0.0,0.0,0.0,0.120065,...,1.286584,0.156455,0.284713,0.047846,0.0,0.237367,0.0,0.0,0.694635,0.0
2,0.561301,0.0,0.160700,0.271133,0.672635,0.0,0.0,0.0,0.0,0.104824,...,1.192837,0.156339,0.272637,0.048556,0.0,0.231901,0.0,0.0,0.756731,0.0
3,0.567952,0.0,0.171620,0.247333,0.679228,0.0,0.0,0.0,0.0,0.095891,...,1.241280,0.160481,0.286083,0.054391,0.0,0.233822,0.0,0.0,0.725959,0.0
4,0.459339,0.0,0.000000,0.145172,0.567070,0.0,0.0,0.0,0.0,0.073279,...,1.165474,0.134485,0.263052,0.000000,0.0,0.231551,0.0,0.0,0.739875,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,0.504626,0.0,0.084883,0.217748,0.620897,0.0,0.0,0.0,0.0,0.117190,...,1.287797,0.159790,0.284527,0.018355,0.0,0.244309,0.0,0.0,0.709146,0.0
295,0.406523,0.0,0.000000,0.092981,0.467883,0.0,0.0,0.0,0.0,0.056216,...,1.171588,0.136378,0.275888,0.052597,0.0,0.237306,0.0,0.0,0.745903,0.0
296,0.511602,0.0,0.110740,0.189782,0.594051,0.0,0.0,0.0,0.0,0.097574,...,1.133468,0.126552,0.267365,0.027393,0.0,0.234932,0.0,0.0,0.731986,0.0
297,0.548836,0.0,0.133466,0.286130,0.609862,0.0,0.0,0.0,0.0,0.121407,...,1.246541,0.150359,0.271137,0.073534,0.0,0.228698,0.0,0.0,0.734360,0.0


In [38]:
#Adding target column

In [39]:
uninfected_df['Target'] = 0

In [40]:
uninfected_df

,0,1,2,3,4,5,6,7,8,9,...,25079,25080,25081,25082,25083,25084,25085,25086,25087,Target
0,0.539940,0.0,0.166913,0.279902,0.653289,0.0,0.0,0.0,0.0,0.128804,...,0.139180,0.265222,0.093659,0.0,0.205243,0.0,0.0,0.701365,0.0,0
1,0.560191,0.0,0.164494,0.302933,0.644150,0.0,0.0,0.0,0.0,0.120065,...,0.156455,0.284713,0.047846,0.0,0.237367,0.0,0.0,0.694635,0.0,0
2,0.561301,0.0,0.160700,0.271133,0.672635,0.0,0.0,0.0,0.0,0.104824,...,0.156339,0.272637,0.048556,0.0,0.231901,0.0,0.0,0.756731,0.0,0
3,0.567952,0.0,0.171620,0.247333,0.679228,0.0,0.0,0.0,0.0,0.095891,...,0.160481,0.286083,0.054391,0.0,0.233822,0.0,0.0,0.725959,0.0,0
4,0.459339,0.0,0.000000,0.145172,0.567070,0.0,0.0,0.0,0.0,0.073279,...,0.134485,0.263052,0.000000,0.0,0.231551,0.0,0.0,0.739875,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,0.504626,0.0,0.084883,0.217748,0.620897,0.0,0.0,0.0,0.0,0.117190,...,0.159790,0.284527,0.018355,0.0,0.244309,0.0,0.0,0.709146,0.0,0
295,0.406523,0.0,0.000000,0.092981,0.467883,0.0,0.0,0.0,0.0,0.056216,...,0.136378,0.275888,0.052597,0.0,0.237306,0.0,0.0,0.745903,0.0,0
296,0.511602,0.0,0.110740,0.189782,0.594051,0.0,0.0,0.0,0.0,0.097574,...,0.126552,0.267365,0.027393,0.0,0.234932,0.0,0.0,0.731986,0.0,0
297,0.548836,0.0,0.133466,0.286130,0.609862,0.0,0.0,0.0,0.0,0.121407,...,0.150359,0.271137,0.073534,0.0,0.228698,0.0,0.0,0.734360,0.0,0


In [41]:
#coverting dict 2D array to 1D array only infected data that is folder 1

In [42]:
infected = np.squeeze(feature_dict_new_ds[1])
infected

array([[0.6866806 , 0.        , 0.31876042, ..., 0.        , 0.65841687,
        0.        ],
       [0.68469733, 0.        , 0.34427676, ..., 0.        , 0.6044284 ,
        0.        ],
       [0.6122143 , 0.        , 0.22844662, ..., 0.        , 0.59160054,
        0.        ],
       ...,
       [0.6802469 , 0.        , 0.32954565, ..., 0.        , 0.59423137,
        0.        ],
       [0.67975414, 0.        , 0.32279956, ..., 0.        , 0.62909955,
        0.        ],
       [0.6367001 , 0.        , 0.29336792, ..., 0.        , 0.52518606,
        0.        ]], dtype=float32)

In [43]:
#storing it in DataFrame

In [44]:
infected_df =  pd.DataFrame(infected)
infected_df

,0,1,2,3,4,5,6,7,8,9,...,25078,25079,25080,25081,25082,25083,25084,25085,25086,25087
0,0.686681,0.0,0.318760,0.240281,0.554595,0.0,0.0,0.0,0.0,0.170058,...,1.228569,0.119431,0.257480,0.662474,0.016617,0.191417,0.0,0.0,0.658417,0.0
1,0.684697,0.0,0.344277,0.269630,0.549066,0.0,0.0,0.0,0.0,0.205106,...,1.193470,0.086857,0.193028,0.942626,0.035129,0.148231,0.0,0.0,0.604428,0.0
2,0.612214,0.0,0.228447,0.059155,0.381893,0.0,0.0,0.0,0.0,0.220718,...,0.963619,0.000000,0.166425,1.293168,0.017087,0.025265,0.0,0.0,0.591601,0.0
3,0.700679,0.0,0.411855,0.335692,0.630833,0.0,0.0,0.0,0.0,0.189966,...,1.301663,0.140057,0.239616,0.339032,0.000000,0.172730,0.0,0.0,0.677015,0.0
4,0.682861,0.0,0.318166,0.315851,0.561206,0.0,0.0,0.0,0.0,0.196463,...,1.113997,0.052931,0.184190,1.282548,0.041094,0.084949,0.0,0.0,0.576096,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,0.565919,0.0,0.167215,0.009551,0.388457,0.0,0.0,0.0,0.0,0.237296,...,1.133460,0.095148,0.200921,0.978239,0.033867,0.164757,0.0,0.0,0.609328,0.0
317,0.683098,0.0,0.325880,0.187747,0.544780,0.0,0.0,0.0,0.0,0.184725,...,1.079589,0.011572,0.164393,1.188645,0.053927,0.093706,0.0,0.0,0.564231,0.0
318,0.680247,0.0,0.329546,0.307149,0.571643,0.0,0.0,0.0,0.0,0.270420,...,1.122101,0.091497,0.137271,1.061857,0.042789,0.128600,0.0,0.0,0.594231,0.0
319,0.679754,0.0,0.322800,0.295100,0.566374,0.0,0.0,0.0,0.0,0.213403,...,1.188861,0.126545,0.237837,0.678645,0.008761,0.180817,0.0,0.0,0.629100,0.0


In [45]:
#Adding target column

In [46]:
infected_df['Target'] = 1
infected_df

,0,1,2,3,4,5,6,7,8,9,...,25079,25080,25081,25082,25083,25084,25085,25086,25087,Target
0,0.686681,0.0,0.318760,0.240281,0.554595,0.0,0.0,0.0,0.0,0.170058,...,0.119431,0.257480,0.662474,0.016617,0.191417,0.0,0.0,0.658417,0.0,1
1,0.684697,0.0,0.344277,0.269630,0.549066,0.0,0.0,0.0,0.0,0.205106,...,0.086857,0.193028,0.942626,0.035129,0.148231,0.0,0.0,0.604428,0.0,1
2,0.612214,0.0,0.228447,0.059155,0.381893,0.0,0.0,0.0,0.0,0.220718,...,0.000000,0.166425,1.293168,0.017087,0.025265,0.0,0.0,0.591601,0.0,1
3,0.700679,0.0,0.411855,0.335692,0.630833,0.0,0.0,0.0,0.0,0.189966,...,0.140057,0.239616,0.339032,0.000000,0.172730,0.0,0.0,0.677015,0.0,1
4,0.682861,0.0,0.318166,0.315851,0.561206,0.0,0.0,0.0,0.0,0.196463,...,0.052931,0.184190,1.282548,0.041094,0.084949,0.0,0.0,0.576096,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,0.565919,0.0,0.167215,0.009551,0.388457,0.0,0.0,0.0,0.0,0.237296,...,0.095148,0.200921,0.978239,0.033867,0.164757,0.0,0.0,0.609328,0.0,1
317,0.683098,0.0,0.325880,0.187747,0.544780,0.0,0.0,0.0,0.0,0.184725,...,0.011572,0.164393,1.188645,0.053927,0.093706,0.0,0.0,0.564231,0.0,1
318,0.680247,0.0,0.329546,0.307149,0.571643,0.0,0.0,0.0,0.0,0.270420,...,0.091497,0.137271,1.061857,0.042789,0.128600,0.0,0.0,0.594231,0.0,1
319,0.679754,0.0,0.322800,0.295100,0.566374,0.0,0.0,0.0,0.0,0.213403,...,0.126545,0.237837,0.678645,0.008761,0.180817,0.0,0.0,0.629100,0.0,1


In [47]:
#concating/merging the two DataFrames into single DataFrame vertically axis=0

In [48]:
mal_df = pd.concat([infected_df, uninfected_df])
mal_df

,0,1,2,3,4,5,6,7,8,9,...,25079,25080,25081,25082,25083,25084,25085,25086,25087,Target
0,0.686681,0.0,0.318760,0.240281,0.554595,0.0,0.0,0.0,0.0,0.170058,...,0.119431,0.257480,0.662474,0.016617,0.191417,0.0,0.0,0.658417,0.0,1
1,0.684697,0.0,0.344277,0.269630,0.549066,0.0,0.0,0.0,0.0,0.205106,...,0.086857,0.193028,0.942626,0.035129,0.148231,0.0,0.0,0.604428,0.0,1
2,0.612214,0.0,0.228447,0.059155,0.381893,0.0,0.0,0.0,0.0,0.220718,...,0.000000,0.166425,1.293168,0.017087,0.025265,0.0,0.0,0.591601,0.0,1
3,0.700679,0.0,0.411855,0.335692,0.630833,0.0,0.0,0.0,0.0,0.189966,...,0.140057,0.239616,0.339032,0.000000,0.172730,0.0,0.0,0.677015,0.0,1
4,0.682861,0.0,0.318166,0.315851,0.561206,0.0,0.0,0.0,0.0,0.196463,...,0.052931,0.184190,1.282548,0.041094,0.084949,0.0,0.0,0.576096,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,0.504626,0.0,0.084883,0.217748,0.620897,0.0,0.0,0.0,0.0,0.117190,...,0.159790,0.284527,0.018355,0.000000,0.244309,0.0,0.0,0.709146,0.0,0
295,0.406523,0.0,0.000000,0.092981,0.467883,0.0,0.0,0.0,0.0,0.056216,...,0.136378,0.275888,0.052597,0.000000,0.237306,0.0,0.0,0.745903,0.0,0
296,0.511602,0.0,0.110740,0.189782,0.594051,0.0,0.0,0.0,0.0,0.097574,...,0.126552,0.267365,0.027393,0.000000,0.234932,0.0,0.0,0.731986,0.0,0
297,0.548836,0.0,0.133466,0.286130,0.609862,0.0,0.0,0.0,0.0,0.121407,...,0.150359,0.271137,0.073534,0.000000,0.228698,0.0,0.0,0.734360,0.0,0


In [49]:
#renaming the column names by adding the prefix 'Feature_'

In [50]:
mal_df=mal_df.add_prefix('Feature_')

In [51]:
#final DataFrame

In [52]:
mal_df

,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,...,Feature_25079,Feature_25080,Feature_25081,Feature_25082,Feature_25083,Feature_25084,Feature_25085,Feature_25086,Feature_25087,Feature_Target
0,0.686681,0.0,0.318760,0.240281,0.554595,0.0,0.0,0.0,0.0,0.170058,...,0.119431,0.257480,0.662474,0.016617,0.191417,0.0,0.0,0.658417,0.0,1
1,0.684697,0.0,0.344277,0.269630,0.549066,0.0,0.0,0.0,0.0,0.205106,...,0.086857,0.193028,0.942626,0.035129,0.148231,0.0,0.0,0.604428,0.0,1
2,0.612214,0.0,0.228447,0.059155,0.381893,0.0,0.0,0.0,0.0,0.220718,...,0.000000,0.166425,1.293168,0.017087,0.025265,0.0,0.0,0.591601,0.0,1
3,0.700679,0.0,0.411855,0.335692,0.630833,0.0,0.0,0.0,0.0,0.189966,...,0.140057,0.239616,0.339032,0.000000,0.172730,0.0,0.0,0.677015,0.0,1
4,0.682861,0.0,0.318166,0.315851,0.561206,0.0,0.0,0.0,0.0,0.196463,...,0.052931,0.184190,1.282548,0.041094,0.084949,0.0,0.0,0.576096,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,0.504626,0.0,0.084883,0.217748,0.620897,0.0,0.0,0.0,0.0,0.117190,...,0.159790,0.284527,0.018355,0.000000,0.244309,0.0,0.0,0.709146,0.0,0
295,0.406523,0.0,0.000000,0.092981,0.467883,0.0,0.0,0.0,0.0,0.056216,...,0.136378,0.275888,0.052597,0.000000,0.237306,0.0,0.0,0.745903,0.0,0
296,0.511602,0.0,0.110740,0.189782,0.594051,0.0,0.0,0.0,0.0,0.097574,...,0.126552,0.267365,0.027393,0.000000,0.234932,0.0,0.0,0.731986,0.0,0
297,0.548836,0.0,0.133466,0.286130,0.609862,0.0,0.0,0.0,0.0,0.121407,...,0.150359,0.271137,0.073534,0.000000,0.228698,0.0,0.0,0.734360,0.0,0


In [53]:
#converting DataFrame into .csv file

In [54]:
mal_df.to_csv('mal_features_svm.csv', index = False)

In [55]:
#created new file

In [56]:
pd.read_csv('mal_features_svm.csv')

,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,...,Feature_25079,Feature_25080,Feature_25081,Feature_25082,Feature_25083,Feature_25084,Feature_25085,Feature_25086,Feature_25087,Feature_Target
0,0.686681,0.0,0.318760,0.240281,0.554595,0.0,0.0,0.0,0.0,0.170058,...,0.119431,0.257480,0.662474,0.016617,0.191417,0.0,0.0,0.658417,0.0,1
1,0.684697,0.0,0.344277,0.269630,0.549066,0.0,0.0,0.0,0.0,0.205106,...,0.086857,0.193028,0.942626,0.035129,0.148231,0.0,0.0,0.604428,0.0,1
2,0.612214,0.0,0.228447,0.059155,0.381893,0.0,0.0,0.0,0.0,0.220718,...,0.000000,0.166425,1.293168,0.017087,0.025265,0.0,0.0,0.591601,0.0,1
3,0.700679,0.0,0.411855,0.335692,0.630833,0.0,0.0,0.0,0.0,0.189966,...,0.140057,0.239616,0.339032,0.000000,0.172730,0.0,0.0,0.677015,0.0,1
4,0.682861,0.0,0.318166,0.315851,0.561206,0.0,0.0,0.0,0.0,0.196463,...,0.052931,0.184190,1.282548,0.041094,0.084949,0.0,0.0,0.576096,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,0.504626,0.0,0.084883,0.217748,0.620897,0.0,0.0,0.0,0.0,0.117190,...,0.159790,0.284527,0.018355,0.000000,0.244309,0.0,0.0,0.709146,0.0,0
616,0.406523,0.0,0.000000,0.092981,0.467883,0.0,0.0,0.0,0.0,0.056216,...,0.136378,0.275888,0.052597,0.000000,0.237306,0.0,0.0,0.745903,0.0,0
617,0.511602,0.0,0.110740,0.189782,0.594051,0.0,0.0,0.0,0.0,0.097574,...,0.126552,0.267365,0.027393,0.000000,0.234932,0.0,0.0,0.731986,0.0,0
618,0.548836,0.0,0.133466,0.286130,0.609862,0.0,0.0,0.0,0.0,0.121407,...,0.150359,0.271137,0.073534,0.000000,0.228698,0.0,0.0,0.734360,0.0,0
